In [1]:
import json

In [6]:
pieces = ["pawn", "knight", "bishop", "rook", "queen", "king", "porn", "on", "night", "ruk"]
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
rows = ['1', '2', '3', '4', '5', '6', '7', '8']

def generate_chess_moves():
    data = []
    for piece in pieces:
        for from_col in columns:
            for from_row in rows:
                for to_col in columns:
                    for to_row in rows:
                        if from_col != to_col or from_row != to_row:  # to avoid moves where piece doesn't move
                            text = f"move {piece} from {from_col}{from_row} to {to_col}{to_row}"
                            data.append({"text": text, "piece": piece, "from": f"{from_col}{from_row}", "to": f"{to_col}{to_row}"})
    return data

# Generate the dataset
data = generate_chess_moves()

# Save to a JSON file
with open('large_chess_moves.json', 'w') as f:
    json.dump(data, f)

print(f"Generated {len(data)} chess moves.")

Generated 40320 chess moves.


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import json

# Load data
with open('large_chess_moves.json') as f:
    data = json.load(f)

# Prepare data
texts = [item['text'] for item in data]
pieces = [item['piece'] for item in data]
from_positions = [item['from'] for item in data]
to_positions = [item['to'] for item in data]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length, padding='post')

# Simplified target encoding
piece_tokenizer = Tokenizer()
piece_tokenizer.fit_on_texts(pieces)
y_pieces = np.array(piece_tokenizer.texts_to_sequences(pieces)).flatten()

position_tokenizer = Tokenizer()
position_tokenizer.fit_on_texts(from_positions + to_positions)
y_from_positions = np.array(position_tokenizer.texts_to_sequences(from_positions)).flatten()
y_to_positions = np.array(position_tokenizer.texts_to_sequences(to_positions)).flatten()

# Reshape the targets to be compatible with model output
y_pieces = y_pieces.reshape(-1, 1)
y_from_positions = y_from_positions.reshape(-1, 1)
y_to_positions = y_to_positions.reshape(-1, 1)

# Build the model using the functional API
input_layer = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
lstm_layer = Bidirectional(LSTM(64))(lstm_layer)

dense_layer = Dense(64, activation='relu')(lstm_layer)

piece_output = Dense(len(piece_tokenizer.word_index) + 1, activation='softmax', name='piece_output')(dense_layer)
from_output = Dense(len(position_tokenizer.word_index) + 1, activation='softmax', name='from_output')(dense_layer)
to_output = Dense(len(position_tokenizer.word_index) + 1, activation='softmax', name='to_output')(dense_layer)

model = Model(inputs=input_layer, outputs=[piece_output, from_output, to_output])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X, [y_pieces, y_from_positions, y_to_positions], epochs=10)

# Save the model and tokenizers
model.save('chess_model.h5')
with open('tokenizer.json', 'w') as f:
    f.write(tokenizer.to_json())
with open('piece_tokenizer.json', 'w') as f:
    f.write(piece_tokenizer.to_json())
with open('position_tokenizer.json', 'w') as f:
    f.write(position_tokenizer.to_json())


Epoch 1/10
1260/1260 [==============================] - 17s 11ms/step - loss: 3.9031 - piece_output_loss: 0.2399 - from_output_loss: 2.2295 - to_output_loss: 1.4337 - piece_output_accuracy: 0.9338 - from_output_accuracy: 0.4374 - to_output_accuracy: 0.6404
Epoch 2/10
1260/1260 [==============================] - 14s 11ms/step - loss: 0.1335 - piece_output_loss: 0.0162 - from_output_loss: 0.0679 - to_output_loss: 0.0495 - piece_output_accuracy: 0.9984 - from_output_accuracy: 0.9961 - to_output_accuracy: 0.9971
Epoch 3/10
1260/1260 [==============================] - 15s 12ms/step - loss: 0.0189 - piece_output_loss: 0.0025 - from_output_loss: 0.0089 - to_output_loss: 0.0075 - piece_output_accuracy: 1.0000 - from_output_accuracy: 1.0000 - to_output_accuracy: 1.0000
Epoch 4/10
1260/1260 [==============================] - 15s 12ms/step - loss: 0.0061 - piece_output_loss: 8.4461e-04 - from_output_loss: 0.0028 - to_output_loss: 0.0024 - piece_output_accuracy: 1.0000 - from_output_accuracy: 1.00